In [ ]:
import pandas as pd
import numpy as np
import os
import re
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
!pip install fsspec==2024.9.0
!pip install datasets

from datasets import Dataset

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/aiffel/Data/DLthon/dataset.csv")
test = pd.read_csv("/content/drive/MyDrive/aiffel/Data/DLthon/test.csv")
sub = pd.read_csv("/content/drive/MyDrive/aiffel/Data/DLthon/submission.csv")

In [ ]:
train_add = pd.read_csv("/content/drive/MyDrive/aiffel/Data/dataset.csv")

In [ ]:
train_add_tail = train_add.tail(600)
train = pd.concat([train, train_add_tail], axis=0).reset_index(drop=True)

In [ ]:
train['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '일반'], dtype=object)

In [ ]:
train.shape

(12353, 4)

In [ ]:
train['class'].value_counts()

,count
class,
일반,3651
기타 괴롭힘 대화,2388
갈취 대화,2162
직장 내 괴롭힘 대화,2158
협박 대화,1994


In [ ]:
train['conversation']

,conversation
0,지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼...
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,어이 거기\n예?? 너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?\...
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...
12348,팀 내 규칙은 지켜야 하지 않겠어요?\n보고서를 왜 이렇게 대충 했습니까?\n무리라...
12349,약속이고 뭐고 일을 먼저 해야 하지 않겠어요?\n죄송하지만 수정할 시간이 부족했어요...
12350,어제 한 약속 지키신 건가요?\n이러면 모두가 힘들어집니다.\n김대리만 자꾸 문제를...
12351,제 시간에 보고서가 안 올라오면 곤란합니다.\n다들 열심히 하는데 혼자만 게으른 건...


In [ ]:
test['text']

,text
0,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...
495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [ ]:
train['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '일반'], dtype=object)

In [ ]:
train['class'].value_counts()

,count
class,
일반,3651
기타 괴롭힘 대화,2388
갈취 대화,2162
직장 내 괴롭힘 대화,2158
협박 대화,1994


In [ ]:
# 데이터 준비
data = train  # train DataFrame
classes = train['class'].unique()

# Train/Validation Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['conversation'], data['class'], test_size=0.2, stratify=data['class'], random_state=42
)

# 레이블 인코딩
label_map = {label: idx for idx, label in enumerate(classes)}
train_labels = train_labels.map(label_map)
val_labels = val_labels.map(label_map)

In [ ]:
# Tokenizer 준비
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512)

# 데이터셋 생성
dataset_train = Dataset.from_dict({
    'text': train_texts.tolist(),
    'label': train_labels.tolist()
})
dataset_val = Dataset.from_dict({
    'text': val_texts.tolist(),
    'label': val_labels.tolist()
})

dataset_train = dataset_train.map(lambda x: tokenize_function(x['text']), batched=True)
dataset_val = dataset_val.map(lambda x: tokenize_function(x['text']), batched=True)

dataset_train = dataset_train.remove_columns(['text'])
dataset_val = dataset_val.remove_columns(['text'])

# PyTorch 데이터셋 준비
dataset_train.set_format('torch')
dataset_val.set_format('torch')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/9882 [00:00<?, ? examples/s]

Map:   0%|          | 0/2471 [00:00<?, ? examples/s]

* 631755f48b419f66cf0b6d3c747f4a1bde491b64

In [ ]:
# 모델 준비
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=len(classes))

# 평가 함수 정의
def compute_metrics(pred):
    predictions, labels = pred
    predictions = torch.argmax(torch.tensor(predictions), dim=1)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"f1": f1}

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # F1-Score로 가장 높은 모델 저장
    save_total_limit=1
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 학습 시작
trainer.train()

# 평가
results = trainer.evaluate()
print("Validation F1 Score:", results["eval_f1"])

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-7eac65ea8d36>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.

Epoch,Training Loss,Validation Loss,F1
1,0.235200,0.199954,0.941392
2,0.086500,0.194113,0.950753
3,0.065000,0.192567,0.954003
4,0.095200,0.196548,0.956796


Validation F1 Score: 0.9567962732276744


In [ ]:
# Classification Report 생성
def compute_classification_report():
    predictions = trainer.predict(dataset_val).predictions
    predictions = torch.argmax(torch.tensor(predictions), dim=1)
    report = classification_report(val_labels, predictions, target_names=classes)
    print(report)

compute_classification_report()

# Test 데이터 예측
def predict_test_data(test):
    test_texts = test['text']
    test_dataset = Dataset.from_dict({
        'text': test_texts.tolist()
    })
    test_dataset = test_dataset.map(lambda x: tokenize_function(x['text']), batched=True)
    test_dataset = test_dataset.remove_columns(['text'])
    test_dataset.set_format('torch')

    predictions = trainer.predict(test_dataset).predictions
    predicted_classes = torch.argmax(torch.tensor(predictions), dim=1).tolist()

    reverse_label_map = {v: k for k, v in label_map.items()}
    predicted_labels = [reverse_label_map[idx] for idx in predicted_classes]

    test['predicted_class'] = predicted_labels
    return test

# Test 데이터 예측 실행
predicted_test = predict_test_data(test)
print(predicted_test[['text', 'predicted_class']])

              precision    recall  f1-score   support

       협박 대화       0.91      0.94      0.93       399
   기타 괴롭힘 대화       0.94      0.92      0.93       478
       갈취 대화       0.94      0.95      0.94       432
 직장 내 괴롭힘 대화       0.96      0.96      0.96       432
          일반       1.00      0.99      1.00       730

    accuracy                           0.96      2471
   macro avg       0.95      0.95      0.95      2471
weighted avg       0.96      0.96      0.96      2471



Map:   0%|          | 0/500 [00:00<?, ? examples/s]

                                                  text predicted_class
0    아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...           갈취 대화
1    우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때?  네? 제가요? ...     직장 내 괴롭힘 대화
2    너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...     직장 내 괴롭힘 대화
3    이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...              일반
4    아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...       기타 괴롭힘 대화
..                                                 ...             ...
495  미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...     직장 내 괴롭힘 대화
496  교수님 제 논문에 제 이름이 없나요?  아 무슨 논문말이야?  지난 번 냈던 논문이...     직장 내 괴롭힘 대화
497  야 너  네 저요? 그래 너 왜요 돈좀 줘봐  돈 없어요 돈이 왜 없어 지갑은 폼이...           갈취 대화
498  야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...     직장 내 괴롭힘 대화
499  엄마 저 그 돈 안해주시면 정말 큰일나요.  이유도 말하지 않고. 몇번째니 경민아....           협박 대화

[500 rows x 2 columns]


In [ ]:
predicted_test[['text', 'predicted_class']]

,text,predicted_class
0,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,갈취 대화
1,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,직장 내 괴롭힘 대화
2,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,직장 내 괴롭힘 대화
3,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,일반
4,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,기타 괴롭힘 대화
...,...,...
495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,직장 내 괴롭힘 대화
496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,직장 내 괴롭힘 대화
497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,갈취 대화
498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,직장 내 괴롭힘 대화


In [ ]:
sub

,file_name,class
0,t_000,갈취 대화
1,t_001,직장 내 괴롭힘 대화
2,t_002,직장 내 괴롭힘 대화
3,t_003,기타 괴롭힘 대화
4,t_004,기타 괴롭힘 대화
...,...,...
495,t_495,직장 내 괴롭힘 대화
496,t_496,직장 내 괴롭힘 대화
497,t_497,갈취 대화
498,t_498,협박 대화


In [ ]:
sub['class'] = test['predicted_class']

In [ ]:
sub['idx'] = sub['file_name']
sub['target'] = sub['class']
sub = sub.drop(columns=['file_name','class'])
sub = sub[['idx', 'target']]
sub['target'] = sub['target'].apply(lambda x: 0 if x == '협박 대화' else
                                                   1 if x == '갈취 대화' else
                                                   2 if x == '직장 내 괴롭힘 대화' else
                                                   3 if x == '기타 괴롭힘 대화' else
                                                   4 if x == '일반' else None)
sub.to_csv('/content/drive/MyDrive/aiffel/Data/DLthon/submission7.csv', index=False, encoding='utf-8-sig')

In [ ]:
sub

,idx,target
0,t_000,1
1,t_001,2
2,t_002,2
3,t_003,4
4,t_004,3
...,...,...
495,t_495,2
496,t_496,2
497,t_497,1
498,t_498,2


In [ ]:
sub['target'].value_counts()

,count
target,
3,130
2,118
1,115
0,102
4,35
